# 04 - Deep MLP
Train the Deep MLP and visualize results.


In [ ]:
from pathlib import Path
import sys

ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd


In [ ]:
from src.models import make_mlp_model, make_pipeline, build_search
from src.eval import evaluate_models
from src.plots import plot_actual_vs_pred, plot_error_distribution
from _common import load_dataset, prepare_features
from src.split import SplitConfig

SEED = 42
TUNE_MODE = "fast"  # off | fast | full
MLP_VERBOSE = 2  # 0/False, 1=tqdm, 2=tqdm + per-epoch log
MLP_LOG_EVERY = 1
MLP_BATCH_LOG_EVERY = 50  # set 0 to disable
MLP_LIVE_PLOT_EVERY = 5   # epochs; set 0 to disable
split_config = SplitConfig(test_rounds=6)
df, metadata = load_dataset()
train_df, val_df, trainval_df, test_df, features = prepare_features(df, metadata, split_config=split_config)

X_train = train_df[features]
y_train = train_df["LapTimeSeconds"].to_numpy()
X_val = val_df[features]
y_val = val_df["LapTimeSeconds"].to_numpy()

base = make_pipeline(make_mlp_model(SEED, verbose=MLP_VERBOSE, log_every=MLP_LOG_EVERY, log_batch_every=MLP_BATCH_LOG_EVERY, live_plot_every=MLP_LIVE_PLOT_EVERY), features)
model = build_search("Deep MLP", base, random_state=SEED, mode=TUNE_MODE)
metrics, preds, fitted = evaluate_models({"Deep MLP": model}, X_train, y_train, X_val, y_val)
metrics


In [ ]:
best = fitted["Deep MLP"].best_estimator_ if hasattr(fitted["Deep MLP"], "best_estimator_") else fitted["Deep MLP"]
X_trainval = trainval_df[features]
y_trainval = trainval_df["LapTimeSeconds"].to_numpy()
X_test = test_df[features]
y_test = test_df["LapTimeSeconds"].to_numpy()
best.fit(X_trainval, y_trainval)
test_pred = best.predict(X_test)

plot_actual_vs_pred(y_test, test_pred, title="Deep MLP: Predicted vs Actual")


In [ ]:
plot_error_distribution(y_test, test_pred, title="Deep MLP: Residuals")


In [ ]:
# Training history (post-hoc)
import plotly.express as px

est = fitted["Deep MLP"]
if hasattr(est, "best_estimator_"):
    est = est.best_estimator_
model = est.named_steps["model"]
hist = getattr(model, "training_history_", None)
if hist:
    df_hist = pd.DataFrame(hist)
    fig = px.line(df_hist, y=["train_loss", "val_loss"], title="Deep MLP Training Curves")
    fig
else:
    print("No training history found.")
